# 📊 DDL: Tabela Silver - `tb_pesquisa`

**Origem:** Tabela Bronze `pesquisa_satisfacao`
**Domínio:** Customer Experience (CX)

## ⚙️ Decisões de Governança e Otimização

### 1. Otimização de Tipagem (SmallInt)
Para métricas discretas e limitadas, evitamos o uso de inteiros longos:
- **`nu_nota`:** O campo `nota` original é convertido para **`SMALLINT`**. Como a escala é fixa (ex: 1 a 5 ou 0 a 10), este tipo de dado consome significativamente menos espaço e é semanticamente mais correto.

### 2. Mapeamento de Schema (Bronze → Silver)
| Coluna Bronze | Coluna Silver | Tipo Novo | Justificativa |
|---|---|---|---|
| `id_pesquisa` | `cd_pesquisa` | `BIGINT` | PK. |
| `id_chamado` | `cd_chamado` | `BIGINT` | FK para `tb_chamado`. |
| `nota` | `nu_nota` | `SMALLINT` | Padronização numérica (`nu_`) e otimização. |

### 3. Constraints
- **Primary Key:** `pk_pesquisa` (`cd_pesquisa`).
- **Regra de Negócio:** Cada chamado pode ter apenas uma pesquisa de satisfação associada (cardinalidade 1:1 ou 0:1).

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import TimestampType

CATALOGO_DESTINO = "v_credit"
SCHEMA_DESTINO = "silver"
TABELA_DESTINO = "tb_pesquisa"

nome_tabela_destino = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_DESTINO}"

In [0]:
query = f"""
CREATE OR REPLACE TABLE {nome_tabela_destino} (
    cd_pesquisa BIGINT COMMENT 'Identificador da pesquisa.',
    cd_chamado BIGINT COMMENT 'Vínculo com o chamado avaliado.',
    nu_nota SMALLINT COMMENT 'Nota atribuída (ex: 1 a 5).',
    dt_ingestion TIMESTAMP,
    dc_origem STRING,
    CONSTRAINT pk_pesquisa PRIMARY KEY (cd_pesquisa)
)
COMMENT 'Tabela silver de histórico de pesquisa de satisfação';
"""

spark.sql(query)
print(f"✅ Tabela {nome_tabela_destino} criada com sucesso!")